In [13]:
from langgraph.graph import StateGraph, START, END
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.tools import tool
from langchain_community.tools import ShellTool
from langgraph.prebuilt import ToolNode
from typing import TypedDict, List
import os
from dotenv import load_dotenv
from github import Github
load_dotenv()

True

In [14]:
# Set up Groq LLM
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")  

llm = ChatGroq(
    model="openai/gpt-oss-20b",
)

In [15]:
# Define the state
class CodeReviewState(TypedDict):
    repo_name: str
    pr_number: int
    code_changes: List[str]
    feedback: List[str]
    issues: List[str]
    suggestions: List[str]

In [10]:
# Define nodes
def analyze_code(state: CodeReviewState):
    code = state["code"]
    prompt = f"""
    Analyze the following code for potential issues, bugs, and improvements.
    Code:
    {code}
    
    Provide a list of issues and suggestions.
    """
    response = llm.invoke([SystemMessage(content="You are a code review expert."), HumanMessage(content=prompt)])
    analysis = response.content
    
    # Parse analysis into issues and suggestions (simple split)
    lines = analysis.split('\n')
    issues = [line for line in lines if 'issue' in line.lower() or 'bug' in line.lower()]
    suggestions = [line for line in lines if 'suggest' in line.lower() or 'improve' in line.lower()]
    
    return {"issues": issues, "suggestions": suggestions}

def provide_feedback(state: CodeReviewState):
    issues = state["issues"]
    suggestions = state["suggestions"]
    feedback = f"Issues: {issues}\nSuggestions: {suggestions}"
    return {"feedback": [feedback]}

In [ ]:
# Fetch PR changes
def fetch_pr_changes(state: CodeReviewState):
    repo_name = state["repo_name"]
    pr_number = state["pr_number"]
    
    g = Github(os.getenv("GITHUB_TOKEN"))
    repo = g.get_repo(repo_name)
    pr = repo.get_pull(pr_number)
    
    files = pr.get_files()
    code_changes = []
    for file in files:
        if file.patch:
            code_changes.append(f"File: {file.filename}\n{file.patch}")
    
    return {"code_changes": code_changes}

# Define nodes
def analyze_code(state: CodeReviewState):
    code_changes = state["code_changes"]
    all_code = "\n\n".join(code_changes)
    prompt = f"""
    Analyze the following code changes for potential issues, bugs, and improvements.
    Code Changes:
    {all_code}
    
    Provide a list of issues and suggestions.
    """
    response = llm.invoke([SystemMessage(content="You are a code review expert."), HumanMessage(content=prompt)])
    analysis = response.content
    
    # Parse analysis into issues and suggestions (simple split)
    lines = analysis.split('\n')
    issues = [line for line in lines if 'issue' in line.lower() or 'bug' in line.lower()]
    suggestions = [line for line in lines if 'suggest' in line.lower() or 'improve' in line.lower()]
    
    return {"issues": issues, "suggestions": suggestions}

def provide_feedback(state: CodeReviewState):
    issues = state["issues"]
    suggestions = state["suggestions"]
    feedback = f"**Code Review Feedback**\n\n**Issues:**\n" + "\n".join(f"- {issue}" for issue in issues) + f"\n\n**Suggestions:**\n" + "\n".join(f"- {sugg}" for sugg in suggestions)
    return {"feedback": [feedback]}

def post_feedback(state: CodeReviewState):
    repo_name = state["repo_name"]
    pr_number = state["pr_number"]
    feedback = state["feedback"][0] if state["feedback"] else "No feedback generated."
    
    g = Github(os.getenv("GITHUB_TOKEN"))
    repo = g.get_repo(repo_name)
    pr = repo.get_pull(pr_number)
    
    pr.create_issue_comment(feedback)
    return {}

In [ ]:
# Build the graph
graph = StateGraph(CodeReviewState)

graph.add_node("fetch_pr", fetch_pr_changes)
graph.add_node("analyze", analyze_code)
graph.add_node("feedback", provide_feedback)
graph.add_node("post", post_feedback)

graph.add_edge(START, "fetch_pr")
graph.add_edge("fetch_pr", "analyze")
graph.add_edge("analyze", "feedback")
graph.add_edge("feedback", "post")
graph.add_edge("post", END)

code_review_agent = graph.compile()

In [ ]:
# Test the agent with PR
# Example: repo_name = "owner/repo", pr_number = 1
result = code_review_agent.invoke({
    "repo_name": "shrivarshapoojari/SmartReview",  # Replace with actual repo
    "pr_number": 1,  # Replace with actual PR number
    "code_changes": [],
    "feedback": [],
    "issues": [],
    "suggestions": []
})
print(result)

{'code': '\ndef hello_world():\n    print("Hello, World!")\n', 'feedback': ["Issues: ['Because the function is so small there are almost no bugs, but there are a handful of style, maintainability, and usability improvements you might consider.', '| # | Issue / Observation | Why it matters | Suggested fix / improvement |']\nSuggestions: ['Because the function is so small there are almost no bugs, but there are a handful of style, maintainability, and usability improvements you might consider.', '| # | Issue / Observation | Why it matters | Suggested fix / improvement |', '### Minimal “improved” version']"], 'issues': ['Because the function is so small there are almost no bugs, but there are a handful of style, maintainability, and usability improvements you might consider.', '| # | Issue / Observation | Why it matters | Suggested fix / improvement |'], 'suggestions': ['Because the function is so small there are almost no bugs, but there are a handful of style, maintainability, and usabi